Get all datasets and the according attribute values which contain (at least) one link without _https_ which also works with the replaced protocol.

In [ ]:
! pip install urlextract

In [1]:
import requests     # 2.18.4
import json         # 2.0.9
import pandas as pd # 0.23.0
import numpy as np
import ast
from urlextract import URLExtract as extract
extractor = extract()

ModuleNotFoundError: No module named 'urlextract'

In [68]:
# BFE Abnahme
packages = 'https://ckan.ogdch-abnahme.clients.liip.ch/api/3/action/package_search?fq=organization:(bundesamt-fur-energie-bfe)&rows=500'

# Make the HTTP request
response = requests.get(packages)

# Use the json module to load CKAN's response into a dictionary
response_dict = json.loads(response.content)

# Check the contents of the response
assert response_dict['success'] is True  # make sure if response is OK

# Get a list of all publications and their information (each publication is a dictionary)
data = response_dict['result']['results'].copy()

# get all upper keys
allKeys = []

# store information about each dataset in a frame
df = pd.DataFrame(data)[['id','title','url', 'relations', 'resources']]
df

In [69]:
def check_url(url):
  try:
    get = requests.head(url)

    if get.status_code < 400:
      return True
    else:
      return False

  except requests.exceptions.RequestException as e:
    raise SystemExit(f"{url}: is NOT reachable \nErr: {e}")

In [70]:
def addHTTPS(url):
  adjusted = url.replace("http:", "https:" )
  adjusted = adjusted.replace("www", "https://www" )

  return adjusted.replace("https://https://www", "https://www" )

In [ ]:
def addHTTP(url):
  adjusted = url.replace("www", "http://www")
  
  return adjusted.replace("http://http://www", "http://www")

In [ ]:
def getURLs(urls):
  
  links = []

  for url in urls:

    url_modified    = ''
    # only store links which are faulty or need a protocol update (https)
    needsToBeListed = False

    if 'https' in url:
      working = check_url(url)

      if not working:
        needsToBeListed = True


    else:
      
      url_modified = addHTTPS(url)
      working = check_url(url_modified)

      if working:
        needsToBeListed = True
        
      else:
        # does not change http: urls
        url_modified = addHTTPS(url)
        working = check_url(url)

        if not working:
          url_modified = ''
          needsToBeListed = True

    if not needsToBeListed:
      continue
    

    links.append([url, url_modified])

  return links


In [ ]:
res = pd.DataFrame(columns=['dataset_name', 'type', 'attribute', 'id', 'old_value', 'new_value'])
res

Test

In [ ]:
cols = ['url', 'relations']

completeList = []

for index, row in df.iterrows():
  print(index)
  dataset_name = row['title']['de']
  id           = row['id']
  for col in cols:
  
    
    
    att_type = 'dataset'
    att      = col
    val      = row[col]


    liste = []


    if col == 'resources':
      att_type = 'resources'
      #change att
      print('nichts')


    else:
      print(val)
      if type(val) == str:
        liste = checkString(val)

      else:
        liste = checkString(str(val))


    if len(liste) == 0:
      continue
    else:
      for elem in liste:
        completeList.append([dataset_name, att_type, att, id, elem[0], elem[1]])


    

--------------------------------------------------------------------------------

In [ ]:
urls = ['http://www.google.ch', 'https://www.google.ch', 'www.google.ch', 'https://data.geo.admin.ch/ch.bfe.windenergie-geschwindigkeit_h150/windenergie-geschwindigkeit_h150/windenergie-geschwindigkeit_h150_2056.fgdb.zip']
markedURLs = checkString(str(urls))

print(len(urls))
print(len(markedURLs))

--------------------------------------------------------------------------------

In [ ]:
def checkString(string):
  print('checkString')

  urls = extractor.find_urls(string)

  # first remove elements which are not actual links
  urls = [url for url in urls if ('http' in url) or ('www' in url)]
  

  return getURLs(urls)

In [ ]:
# for a string input
def fromString(string):
     
     needsToBeListed = False
     urls = extractor.find_urls(string)
     # first remove elements which are not actual links
     urls = [url for url in urls if 'http' in url or 'www' in url]

     modified_string = '-'
     
     for url in urls:
          
          print(url)

          working      = False
          url_modified = ''
          
          if 'https://' in url:
            working = check_url(url)

            if not working:
              needsToBeListed = True

          # change to https://
          else:
            url_modified = addHTTPS(url)
            working = check_url(url_modified)

            # adds links which work with https
            if working:
              needsToBeListed = True
            
            # check if the links work with http
            else:
              url_modified = addHTTP(url)
              working = check_url(url)

          
          if working:
            modified_string = string.replace(url, url_modified)

          print(modified_string)
          print()
               
     print()
     if needsToBeListed:
    
          return [string, modified_string]
     else:
          return []

In [ ]:
def fromDictionary(d):

    changed  = False

    for key in d.keys():

        val      = d[key]
        isString = True
        

        if type(val) not in [dict, str, list]:
            continue
        elif type(val) != str:
            isString = False
            val = str(val)

        urls = extractor.find_urls(val)

        if len(urls) == 0:
            continue
        else:
            for url in urls:

                working = False

                if not any(prefix in url for prefix in ['www', 'http']):
                    continue
                elif 'https://' in url:
                    continue
                else:
                    #first check if the adjusted url works, then replace it in the original

                    working, adjusted_url = testAdjustedURL(url)

                    if working:
                        changed = True

                        if isString:
                            d[key] = val.replace(url, adjusted_url)
                        else:
                            d[key] = ast.literal_eval(val.replace(url, adjusted_url))


    if changed:
        return [d]
    else:
        return []